# Synthetic Control Method
This notebook is part of the Research Project done by Jefry el Bhwash as a final project in the MSc. Track: Applied Data Science.
It was done in cooperation with Utrecht University and Nederlandse Publieke Omroep.

It involves the steps taken to implement the Synthetic Control Method to measure the impact of a feature release on user engagement.

Because this was done in cooperation with a company any market sensitve information was removed from the outputs. This makes it easy for an employee to re-run the code and see the information, while hiding it from others who are interested in the implementation.
This is the reason why some entire code-blocks are commented.








In [ ]:
# @title Variables of interest
start_date = '2024-03-15' # @param {type:"date"}
end_date = '2024-05-22' # @param {type:"date"}
date_of_feature = "2024-04-10" # @param {type:"date"}
feature_of_interest = 'ectr' # @param {type:"string"}
percentage_cutoff = 0.5 # @param {type:"number"}

In [ ]:
#Imports required to run this project

#BigFrames
import bigframes.pandas as bf
#Variables of Importance
bf.options.bigquery.location = "EU" #this variable is set based on the dataset you chose to query
bf.options.bigquery.project = "datamart-ads-students-2024" #this variable is set based on the dataset you chose to query

#Standard
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime

# SCM
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import (check_X_y, check_array, check_is_fitted)
from sklearn.metrics import r2_score
import cvxpy as cp
from toolz import partial
from scipy.optimize import minimize


#Visualizations
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import missingno as msno
import matplotlib.ticker as ticker

In [ ]:
#Settings for matplotlib for plots in the thesis
font = {'family' : 'sans-serif',
        'weight' : 'regular',
        'size'   : 11}

mpl.rc('font', **font)
plt.rcParams["svg.fonttype"] = 'path'

In [ ]:
#This is a dictionary to create a sankey diagram of the processing steps
sankeyData = {}

# Dataset

In [ ]:
#SQL Query to access the database
outcome_query_ECTR = """
SELECT
  DATE(E.logged_timestamp_minute) as timestamp_date,
  E.subscription,
  E.platform_type,
  S.operating_system,
  COUNT(E.rec_panel_location) AS count_location,
  COUNT(E.has_view) as total_view,
  AVG(E.total_minutes_watched) as avg_total_mins_watched,
  AVG(S.streamduration_seconds) as avg_streaming_duration_seconds,
  MIN(E.total_minutes_watched) as min_total_minutes,
  MAX(E.total_minutes_watched) as max_total_minutes,
  SUM(CASE WHEN E.total_minutes_watched>=1.0 THEN 1 ELSE 0 END) as engaged_views,
  COUNT(E.has_view)/COUNT(E.rec_panel_location) as ctr,
  SUM(CASE WHEN E.total_minutes_watched>=1.0 THEN 1 ELSE 0 END)/ NULLIF(COUNT(E.has_view),0) as ectr --defined similarly as the dashboard table

FROM `datamart-ads-students-2024.npo_intermediary_topspin_engaged_impressions.v1` AS E
LEFT JOIN `datamart-ads-students-2024.npo_intermediary_topspin_streams_daily.v1` AS S
ON E.session_id = S.session_id

WHERE
  DATE(E.logged_timestamp_minute) >= "2024-01-01" AND
  DATE(E.logged_timestamp_minute) <= "2024-05-30" AND
  DATE(S.startdate_streamstart) >= "2024-01-01" AND
  DATE(S.startdate_streamstart) <= "2024-05-30" AND
  E.rec_panel_location = "home"

GROUP BY timestamp_date, E.platform_type, S.operating_system, E.subscription
"""

#SQL Query to retrieve the information above to be more efficient in terms of
#processing speed, cost and power.
outcome_query_preRun = """
SELECT *  FROM `datamart-ads-students-2024.JB_npo_intermediary_topspin_pagepaths_and_streams.januari-may`
"""

data = bf.read_gbq(outcome_query_preRun)
# data.head()

In [ ]:
# Creating a copy of the data as a regular pandas dataframe
df = data.to_pandas().copy(deep=False)

In [ ]:
#Changing the date to datetime format from PyArrow format
df['timestamp_date'] = pd.to_datetime(df['timestamp_date'], format='%Y/%m/%d')
df = df.rename(columns={"timestamp_date": "date"})

#Sorting the dataframe
df = df.sort_values("date")

# print("Description")
# display(df.describe())
# print("\n\nHead")
# display(df.head())

## Exploratory Data Analysis and Data Cleaning

In [ ]:
#SANKEY
sankeyData['complete'] = df['count_location'].sum()

In [ ]:
#FILTER: Data range filtering - so this does not further implicate the exploratory data analysis
df = df[df['date'] > start_date]
df = df[df['date'] < end_date]

In [ ]:
#SANKEY
sankeyData['datefilter'] = df['count_location'].sum()

The SCM will use certain charachteristics to dicsern between different groups. These group indications will be derived from multiple attributes.

**Device type:**
> Grouping by Operating System and Platform type, for example app users of iOS. This represents the devices that were logged.

**Access type:**
>Grouping by device type and subscription. This ultimately will be used to split the group in donor pool units and treatment units.

In [ ]:
#Creating a single descriptive column for the group indicators
df['access_type'] = df['platform_type'] + '_' + df['operating_system'] + '_' + df['subscription']
df["device_type"] = df['platform_type'] + '_' + df['operating_system']

### Device Type Distribution

In [ ]:
#Plotting the amount of users that each device type holds
# fig = px.box(df,x="device_type", y="count_location", color="platform_type", title="Amount of users each day by device type")
# fig.show()

In [ ]:
# Altered plot

# Group by 'date', 'device_type', and 'platform_type' to sum 'count_location'
grouped_df = df.groupby(['date', 'device_type', 'platform_type'])['count_location'].sum().reset_index()

# Calculate the total count per day
total_per_day = grouped_df.groupby('date')['count_location'].sum().reset_index()
total_per_day.rename(columns={'count_location': 'total_count'}, inplace=True)

# Merge the total count per day back to the grouped_df
merged_df = pd.merge(grouped_df, total_per_day, on='date')

# Calculate the percentage of each device type and platform type per day
merged_df['percentage'] = (merged_df['count_location'] / merged_df['total_count']) * 100

# Create the box plot
fig = px.box(
    merged_df,
    x="device_type",
    y="percentage",
    color="platform_type",
    title="Percentage of users each day by device type and platform type (Aggregated)",
    labels={
        "device_type": "Device Type",
        "percentage": "Percentage of Users",
        "platform_type": "Platform Type"
    }
)

# Update y-axis to show percentages
fig.update_layout(yaxis_ticksuffix="%")

# Update hover template to show percentages with 2 decimal places
fig.update_traces(
    hovertemplate="<br>".join([
        "Device Type: %{x}",
        "Platform Type: %{color}",
        "Percentage: %{y:.2f}%",
        "<extra></extra>"
    ])
)

# Show the plot
fig.show()

In [ ]:
# fig = px.box(df, x="device_type", y="count_location", color="platform_type",
#              facet_col="platform_type",
#              facet_col_wrap=2,
#              facet_col_spacing=0.08#,  # Facet by platform_type
#              ) # title="Amount of users each day by device type (Facet by Platform)"
# fig.update_yaxes(matches=None, showticklabels=True, title="Users per day")
# fig.update_xaxes(matches=None,tickangle=45)
# fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
# # hide subplot y-axis titles and x-axis titles
# for axis in fig.layout:
#     if type(fig.layout[axis]) == go.layout.XAxis:
#         fig.layout[axis].title.text = ''
# fig.show()

#### Average Device types per day

In [ ]:
# Group by 'date' and 'device_type' to sum 'count_location'
grouped_df = df.groupby(['date', 'device_type'])['count_location'].sum().reset_index()

# Calculate the total count per day
total_per_day = grouped_df.groupby('date')['count_location'].sum().reset_index()
total_per_day.rename(columns={'count_location': 'total_count'}, inplace=True)

# Merge the total count per day back to the grouped_df
merged_df = pd.merge(grouped_df, total_per_day, on='date')

# Calculate the percentage of each device type per day
merged_df['percentage'] = (merged_df['count_location'] / merged_df['total_count']) * 100

# Group by 'device_type' and calculate the average percentage
average_percentage_df = merged_df.groupby('device_type')['percentage'].mean().reset_index()

# Sort the DataFrame by 'percentage'
average_percentage_df = average_percentage_df.sort_values(by='percentage', ascending=False)

# Plot using Plotly
fig = px.bar(average_percentage_df, x='device_type', y='percentage',
             title='Average Percentage of Device Types Used',
             labels={'percentage': 'Average Percentage', 'device_type': 'Device Type'},
             category_orders={'device_type': average_percentage_df['device_type'].tolist()})

fig.show()

In [ ]:
#THESIS PLOT
# Plot the bars.
fig, ax = plt.subplots()
x_ticks = list(range(len(average_percentage_df['percentage'])))
ax.bar(average_percentage_df['device_type'], average_percentage_df['percentage'], color='#7a7a7a', width=0.6, zorder=1)

# Remove axis lines.
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

# Add labels to x axis.
ax.set_xticks(x_ticks)
ax.set_xticklabels(average_percentage_df['device_type'])
ax.tick_params(axis='x', labelrotation=90)

# Add labels to y axis.
y_ticks = [1,5, 10, 15,20,25,30]
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_ticks)
ax.yaxis.set_major_formatter(ticker.PercentFormatter(decimals=0))

# Remove tick marks.
ax.tick_params(
    bottom=False,
    left=False
)

ax.set_axisbelow(False)
# Add bar lines as a horizontal grid.
ax.yaxis.grid(color='white', zorder=2)

plt.savefig("deviceTypeDist.svg")

Because some groups are representing less than a percent of the userbase it might be wise to remove them from the analysis. This is done because the behaviour of the little amount of users from a single platform will be given the same weight inititally. Therefore the choice is made to remove them from the analysis.

In [ ]:
#FILTER: Keep only device types where more than `percentage_cutoff` of users are residing
deleted_device_types = list(average_percentage_df[average_percentage_df['percentage'] < percentage_cutoff]["device_type"])
print(f"Removed Devices: {deleted_device_types}")
df = df[~df['device_type'].isin(deleted_device_types)]

In [ ]:
#SANKEY
sankeyData['deviceTypeFilter'] = df['count_location'].sum()

## Missing Data

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

#Normal Matrix
msno.matrix(df.set_index("date"), sparkline=False, ax=axes[0], fontsize=12)
axes[0].set_title('Matrix of missing values')
#Also 0 as missing
msno.matrix(df.replace(0, np.nan).set_index("date"), sparkline=False, ax=axes[1], fontsize=12)
axes[1].set_title('Matrix of missing- and 0-values')

plt.tight_layout()
plt.show()

In [ ]:
# df

Enaged view and total view are sometimes 0, causing the ectr which is calculated on those to be NA, since division by 0 is impossible.

### Missing data imputation
replacing each missing value with the mean of the column

In [ ]:
#Impute based on the mean
na_cols = df.columns[(df == 0).any() | (df == 1).any()]
df[na_cols] = df[na_cols].replace({0: np.nan, 1: np.nan})

# Iterate through columns
for col in df.columns:
    if df[col].dtype == 'Float64':  # Replace NaN with mean for float columns
        df[col].fillna(df[col].mean(), inplace=True)
    elif df[col].dtype == 'Int64':  # Replace NaN with mode for int columns
        df[col].fillna(df[col].mean().astype(int), inplace=True)

In [ ]:
# df.describe()

In [ ]:
# #SANKEY
# sankeyData['imputation'] = df['count_location'].sum()

## Extracting Information
We need to extract information from the data to find the treatment and control units. We also might want to use information like people accessing the platform through a website or through an application, since these would probably result in different behaviours.

In [ ]:
def checkAccountHolder(entry):
  '''
  checkAccountHolder(entry) is a function that determines
  if a stream was seen with an account or without one

  `entry` is the row of the dataframe

  Anonymous should be 0
  accounts like free and premium should be 1
  if the account type is not one of those -1 is entered
  '''
  if entry["subscription"] == "anonymous":
    account = 0
  elif entry["subscription"] == "free":
    account = 1
  elif entry["subscription"] == "premium":
    account = 1
  elif entry["subscription"] == "plus":
    account = 1
  else:
    account = -1
  return account


#Applying the above function checkAccountHolder
#   to get part of the criteria for the intervention group
df['account_holder'] = df.apply(checkAccountHolder, axis=1)

In [ ]:
#Weekend Variable
# 1 if weekend (sat, sun) [5,6]
# 0 if weekday (mom, tue, wed, thu, fri) [0,1,2,3,4]

def weekend_logic(row):
  '''
  A function that returns 1 if the day of the week is
  fri, sat, sun
  '''
  if row['date'].weekday() > 4:
    weekend = 1
  elif row['date'].weekday() < 5:
    weekend = 0
  else:
    weekend = -1
  return weekend

df["weekend"] = df.apply(weekend_logic, axis=1)

In [ ]:
#Website Variable
# Only 1 if the user is using the website through a desktop or laptop, not through a phone.
website_OS = ["Mac OS X", "Windows", "Linux", "Ubuntu", "Chrome OS", "WebOS"]
def website_logic(row):
  '''
  A function that returns 1 if the platform was website
  '''
  if row['platform_type'] == "site" and row["operating_system"] in website_OS:
    site = 1
  else:
    site = 0
  return site

df["site"] = df.apply(website_logic, axis=1)

In [ ]:
# df.head()

## Dataframe preparation for the Synthetic Control Model


Intervention Group:

>Feature: Add or Remove followed shows

Platform to the feature:
* `Web`: 2024-04-10
* `Android TV`: 2024-05-22
* `Android Mobile` 2024-05-22
* `iOS mobile` & `Apple TV`: NA

So if data is taken till 2024-05-22, which is enough time to measure the effect. We can include Android and iOS users as well.

**intervention group:** web users with an account

**donor pool:** app users with an account and all anonymous users

In [ ]:
# Create a column that is called Intervention based on the following criteria:
#   platform = site
#   account holder = 1
#   otherwise 0 - this will be our control group

def intervention_attribute(row):
  '''
  intervention_attribute(row) is a function that checks
  if a user is from the group that has gotten the intervention
  '''
  if row["platform_type"] == "site" and row["account_holder"] == 1:
    intervention = 1 #people with an account using the website platform
  elif row["platform_type"] == "app" and row["account_holder"] == 1:
    intervention = 0 #people with an account using the applications
  elif row["subscription"] == "anonymous":
    intervention = 0 #anonymous users of all platforms
  else:
    intervention = -1 #people with an account on different platforms
  return intervention


#Applying the intervention function to be able to split the groups
df['intervention'] = df.apply(intervention_attribute, axis=1)

In [ ]:
#Creating a new dataframe based on the intervention group
df_scm = df.loc[df['intervention']!=-1]

In [ ]:
# df_scm.head()

In [ ]:
print(f"Rate of site users in this dataframe: {df_scm['site'].sum()/df_scm['site'].shape[0]}")

# Synthetic Control Method
The method is very well explained in different resources.
>In short we are going to emulate an **Random Control Trial** by synthetically recreating a control group using the groups we have assigned to the donor pool.

The book [Facure, 2023] has been especially helpfull in translating the (sometimes daunting) mathematics of the artical [Abadie, 2021] to the Python scripts we find below. Almost all of the scripts mentioned in the book don't work immediately so they have been altered to work for this application, while keeping the original work of Abadie in mind.

## Splitting the Data

In [ ]:
# This function splits the data in 4 sets:
#  1) Pre-intervention Control
#  2) Pre-intervention treatment
#  3) post intervention Control
#  4) Post intervention Treatment

def reshape_sc_data_original(df: pd.DataFrame, #Dataframe
                    geo_col: str, #Group definition
                    time_col: str, #Time column
                    y_col: str, #Variable of interest
                    tr_geos: str, #Treatment definition
                    tr_start: str): #Treament date

  #Pivotting the dataframe
  df_pivot = df.pivot_table(index=time_col, columns=geo_col,values= y_col)

  #Only selecting columns that are present in the dataframe
  tr_geos = list(set(df_pivot.columns) & set(tr_geos))

  #Splitting the pivoted group in Control and Treatment units
  y_co = df_pivot.drop(columns=tr_geos).dropna(axis=1)
  y_tr = df_pivot[tr_geos].dropna(axis=1)

  #Splitting the group by pre-intervention times and post intervention times
  y_pre_co = y_co[df_pivot.index < tr_start]
  y_pre_tr = y_tr[df_pivot.index < tr_start]

  y_post_co = y_co[df_pivot.index >= tr_start]
  y_post_tr = y_tr[df_pivot.index >= tr_start]

  return y_pre_co, y_pre_tr, y_post_co, y_post_tr

In [ ]:
#Creating a list that contains the access_types of all the treatment unit groups
treatment_units = list(df_scm.query("intervention==1")["access_type"].unique())

#Split groups
y_pre_co, y_pre_tr, y_post_co, y_post_tr = reshape_sc_data_original(df = df_scm,
                                                            geo_col = "access_type",
                                                            time_col = "date",
                                                            y_col = feature_of_interest,
                                                            tr_geos = treatment_units,
                                                            tr_start = date_of_feature)
# display(y_pre_tr.head())

### Visualizing the split data
The treatment group exists of multiple device and access types.
Choosing between the mean of all of these or picking a single access_type will be done based on the visualization below.

In [ ]:
# #Figure of control vs treatment

# # Data preparation
# df_pivot = df_scm.pivot_table(index='date', columns='access_type',values= feature_of_interest)

# treatment_unit_droplist = list(set(df_pivot.columns) & set(treatment_units)) #making sure these columns are in the dataframe

# ## Splitting in control and treatment units
# y_co = df_pivot.drop(columns=treatment_unit_droplist) #dropping the treatment units
# y_tr = df_pivot[treatment_unit_droplist] #keeping the treatment units

# # Figure building
# fig = go.Figure()

# # The control group
# for column in y_co.columns:
#     fig.add_trace(go.Scatter(
#         x=y_co.index,
#         y=y_co[column],
#         mode='lines',
#         name=f'{column}',
#         line=dict(color="orange", dash='dash'),
#         opacity=1
#     ))
# # The treatment group
# for column in y_tr.columns:
#     fig.add_trace(go.Scatter(
#         x=y_tr.index,
#         y=y_tr[column],
#         mode='lines',
#         name=f'{column}',
#         line=dict(color="blue", dash='dash'),
#         opacity = 1
#     ))

# # The mean of the treatment group
# fig.add_trace(go.Scatter(
#     x=y_tr.index,
#     y=y_tr.mean(axis=1),
#     mode='lines+markers',
#     name='Mean of treatment Group',
#     line=dict(color='black', width=2)
# ))

# # Change range and title of the plot
# fig.update_layout(
#     yaxis=dict(range=[0, 1]),
#     title="ECTR per Access type"
# )

# fig.show()

In [ ]:
# #THESIS PLOT

# # Data preparation
# df_pivot = df_scm.pivot_table(index='date', columns='access_type',values= feature_of_interest)

# treatment_unit_droplist = list(set(df_pivot.columns) & set(treatment_units)) #making sure these columns are in the dataframe

# ## Splitting in control and treatment units
# y_co = df_pivot.drop(columns=treatment_unit_droplist) #dropping the treatment units
# y_tr = df_pivot[treatment_unit_droplist] #keeping the treatment units


# # Function to plot a series, skipping NaN values
# def plot_series(ax, x, y, color, label, alpha=1.0):
#     mask = ~np.isnan(y)
#     ax.plot(x[mask], y[mask], color=color, label=label, alpha=alpha)

# # Create subplot
# fig, ax = plt.subplots(figsize=(12, 6))

# # Plot y_co dataframe columns in orange
# # for column in y_co.columns:
# #     plot_series(ax, y_co.index, y_co[column], 'tab:orange', f'y_co_{column}', alpha=0.3)

# # # Plot y_tr dataframe columns in blue
# for column in y_tr.columns:
#     plot_series(ax, y_tr.index, y_tr[column], 'tab:blue', f'y_tr_{column}', alpha=0.3)

# # Calculate and plot the mean of y_tr in black
# y_tr_mean = y_tr.mean(axis=1)
# plot_series(ax, y_tr.index, y_tr_mean, 'black', 'y_tr_mean')

# #DateLine
# plt.axvline(datetime.datetime(2024,4,10), color = 'black', linestyle = 'dashed')
# # Customize the plot
# ax.set_title('The Treatment Units')
# ax.set_ylabel('ECTR')
# ax.set_xlim([datetime.datetime(2024,3,16), datetime.datetime(2024,5,21)])
# ax.set_ylim(0,1)
# plt.xticks(rotation=45, ha='right')

# # Show the plot
# plt.tight_layout()
# # plt.savefig("treatmentMeanAndDonor.svg")
# plt.savefig("treatmentMeanAndTreatments.svg")
# plt.show()

#### Exclusion of some groups
As mentioned in [Abadie, 2021] in the Contextual Requirements chapter it is important to have a comparison group available;

> “Moreover, it is important to restrict the donor pool to units with characteristics that are similar to the affected unit. The reason is that, while the restrictions placed on the weights, W, do not allow extrapolation, interpolation biases may still be important if the synthetic control matches the characteristics of the affected unit by averaging away large discrepancies between the characteristics of the affected unit and the characteristics of the units in the synthetic control.”
>
>&mdash; <cite>(Abadie, 2021, p. 409)</cite>

In [ ]:
#These groups are removed because they behave very differently from the other groups
manual_exclusion = ['app_Android_plus',
                    'app_Chromecast_plus',
                    'tvapp_Android_anonymous',
                    'site_Mac OS X_free',
                    'site_Chrome OS_free',
                    'site_Chrome OS_premium',
                    'site_iOS_premium',
                    'site_Android_premium',
                    'site_iOS_free',
                    'site_Android_free']

In [ ]:
#SANKEY
sankeyData['manualExclusion'] = df[~df['access_type'].isin(manual_exclusion)]['count_location'].sum()

In [ ]:
#Split groups
y_pre_co, y_pre_tr, y_post_co, y_post_tr = reshape_sc_data_original(df = df_scm[~df_scm['access_type'].isin(manual_exclusion)],
                                                            geo_col = "access_type",
                                                            time_col = "date",
                                                            y_col = feature_of_interest,
                                                            tr_geos = treatment_units,
                                                            tr_start = date_of_feature)
# display(y_pre_tr.head())

## Model
This section will describe the creation of the synthetic control using the donor pool units that were assigned above.



In [ ]:
# @title Setting the Treatment to Mean or Single Access Type
y_pre_tr_variable = y_pre_tr.mean(axis=1) # @param ["y_pre_tr.mean(axis=1)", "y_pre_tr[\"site_Windows_premium\"]"] {type:"raw"}
y_post_tr_variable = y_post_tr.mean(axis=1) # @param ["y_post_tr.mean(axis=1)", "y_post_tr[\"site_Windows_premium\"]"] {type:"raw"}

In [ ]:
#The Synthetic Control Method class
class SyntheticControl(BaseEstimator, RegressorMixin):
  def __init__(self,):
    pass

  #This fit function determines the weights for the units.
  def fit(self, y_pre_co, y_pre_tr):

    #This is not required, but checks the dimensions of X and Y.
    y_pre_co, y_pre_tr = check_X_y(y_pre_co, y_pre_tr)

    #Initializing the weight vector.
    w = cp.Variable(y_pre_co.shape[1])

    #The objective is to minimize the error, similar to OLS
    #  However there are some constraints put in place, making it different.
    objective = cp.Minimize(cp.sum_squares(y_pre_co@w - y_pre_tr))

    #These constraint make sure there will be no extrapolation.
    constraints = [cp.sum(w) == 1, w >= 0]

    problem = cp.Problem(objective, constraints)

    self.loss_ = problem.solve(verbose=False)
    self.w_ = w.value

    self.is_fitted_ = True
    return self


    def predict(self, y_co):

      check_is_fitted(self)
      y_co = check_array(y_co)

      return y_co @ self.w_

In [ ]:
# Apply the model
model = SyntheticControl()
model.fit(y_pre_co, y_pre_tr_variable)

#the weights are stored in w_
model.w_.round(3)

### Fit of the Model

In [ ]:
y0_tr_hat     = y_post_co.dropna(axis=1).dot(model.w_) #Synthetic Control (=donor pool post treatment X its weights)
y0_tr_hat_fit = y_pre_co.dropna(axis=1).dot(model.w_) #Fit of the synthetic control (=donor pool pre treatment X its weights)

In [ ]:
print(f"R2 of the fit: {round(r2_score(y_pre_tr_variable, y0_tr_hat_fit),3)}")

#Combined Plot
plt.figure(figsize=(10,5))


#PreTreatment
plt.plot(y_pre_tr_variable.index, y_pre_tr_variable.values, label="Treatment Group", color='tab:blue')
plt.plot(y0_tr_hat_fit.index, y0_tr_hat_fit.values, linestyle='dashed', label="Fit of the method", color='tab:red', alpha=0.5)

#PostTreatment
plt.plot(y_post_tr_variable.index, y_post_tr_variable.values, color='tab:blue')
plt.plot(y0_tr_hat.index, y0_tr_hat.values, label="Synthetic Control", linestyle='dashed', color='tab:orange', alpha=1)

#DateLine
plt.axvline(datetime.datetime(2024,4,10), color = 'black', linestyle = 'dashed')

# plt.xlabel('Date')
plt.ylabel(feature_of_interest)


plt.title('ECTR\nThe Synthetic Control and Intervention Group')
plt.legend(loc="lower right")
plt.grid(axis='y')
plt.ylim(0,0.6)
plt.xlim([datetime.datetime(2024,3,16), datetime.datetime(2024,5,21)])
plt.tight_layout()
plt.xticks(rotation=45, ha='right')


plt.savefig("ECTR_normalFit.svg")
# plt.show()

## Adding a Covariate
The model could use more information than solely the ECTR of each unit. We are going to see if adding the variable "site" to the model. Site is chosen because it offers the model information about the behaviour of the unit. Since as established earlier, we expect that the desktop/laptop usage will be different than other types of usage.

In [ ]:
#Adding a variable in long format
reshaper = partial(reshape_sc_data_original,
                    df=df_scm,
                    geo_col="access_type",
                    time_col="date",
                    tr_geos=treatment_units,
                    tr_start="2024-04-10")

y_pre_co, y_pre_tr, y_post_co, y_post_tr = reshaper(
    y_col=feature_of_interest
)

x_pre_co, _, x_post_co, _ = reshaper(y_col="site")

In [ ]:
# A function that finds the optimal fit of the model,
# based on the weights of the two covariates
def find_w_given_vs(vs, x_co_list, y_tr_pre):

    #X are the covariates, v is the weight assigned to the covariates
    X_times_v = sum([x*v for x, v in zip(x_co_list, vs)])

    model = SyntheticControl()
    model.fit(X_times_v, y_tr_pre)

    return {"loss": model.loss_, "w": model.w_}

In [ ]:
def v_loss(vs):
  return find_w_given_vs(vs,
                        [y_pre_co, x_pre_co],
                         y_pre_tr_variable).get("loss")

v_solution = minimize(v_loss, [0, 0], method='L-BFGS-B')
v_solution.x

In [ ]:
#The original Weights by implying that the importance of the second feature is 0
find_w_given_vs([1, 0],
                [y_pre_co, x_pre_co],
                y_pre_tr_variable).get("w").round(3)

In [ ]:
#New weights for the donor pool units, when adding the new covariate
w_cov = find_w_given_vs(v_solution.x,
                        [y_pre_co, x_pre_co],
                        y_pre_tr_variable).get("w").round(3)
w_cov

In [ ]:
y0_hat_cov = sum([x*v for x, v in zip([y_post_co, x_post_co], v_solution.x)]).dot(w_cov)
y0_hat_fit_cov =sum([x*v for x, v in zip([y_pre_co, x_pre_co], v_solution.x)]).dot(w_cov)

In [ ]:
print(f"R2 of the fit: {round(r2_score(y_pre_tr_variable, y0_hat_fit_cov),3)}")
#Combined Plot
plt.figure(figsize=(10, 6))

#PreTreatment
plt.plot(y_pre_tr_variable.index, y_pre_tr_variable.values, label="Treatment Group", color='tab:blue', marker=".")
plt.plot(y0_hat_fit_cov.index, y0_hat_fit_cov.values, label="Fit of Synthetic control on Treatment", linestyle='dashed', color='tab:red', alpha=0.5)

#PostTreatment
plt.plot(y_post_tr_variable.index, y_post_tr_variable.values, marker=".")
plt.plot(y0_hat_cov.index, y0_hat_cov.values, label="Synthetic Control", linestyle='dashed', color='tab:orange', alpha=0.7)

#DateLine
plt.axvline(datetime.datetime(2024,4,10), color = 'black', linestyle = 'dashed')

# plt.xlabel('Date')
plt.ylabel(feature_of_interest)

plt.title('ECTR \nThe Synthetic Control and Intervention Group with covariate: site')
plt.legend(loc="lower right")
plt.ylim(0,0.6)
plt.xlim([datetime.datetime(2024,3,16), datetime.datetime(2024,5,21)])
plt.grid(axis='y')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.savefig("ECTR_covariateFit.svg")
# plt.show()

### Effect of the Feature on the treatment Variable

In [ ]:
plt.plot(y_pre_tr_variable-y0_hat_fit_cov, color='black')
plt.plot(y_post_tr_variable-y0_hat_cov, color='black', linestyle='dashed')

#Indicative Lines
plt.axvline(datetime.datetime(2024,4,10), color = 'black', linewidth=0.5)
plt.axhline(0, color = 'black', linewidth=0.5)

plt.xlim([datetime.datetime(2024,3,16), datetime.datetime(2024,5,21)])
plt.ylim([-0.1,0.1])
plt.title("Difference between synthetic control and treatment unit\n(with site)")
plt.tight_layout()
plt.tick_params(axis='x', labelrotation=45)
plt.savefig("Effect_covariateFit.svg")
# plt.show()

## Preventing Overfitting by using K-fold Cross Validation
If the donor pool units are getting close to overfitting, we could run into bias issues. Therefore it is safer to prevent the overfitting by applying K-fold cross validation on the training units.


- ATT: Average Treatment effect of the Treated

In [ ]:
#Disclaimer: This function does not use the added covariate
def debiased_sc_atts(y_pre_co, y_pre_tr, y_post_co, y_post_tr, K=3):
    #How many days in a single block
    block_size = int(min(np.floor(len(y_pre_tr)/K), len(y_post_tr)))
    #The indexes of each block
    blocks = np.split(y_pre_tr.index[-K*block_size:], K)

    #The resulting lists
    debiased_effects = []
    block_weights = []

    def fold_effect(hold_out):
        #The same model
        model = SyntheticControl()
        #Fit the model on the data without one of the blocks
        model.fit(y_pre_co.drop(hold_out), y_pre_tr.drop(hold_out))

        #An estimate for the bias
        bias_hat = np.mean(y_pre_tr.loc[hold_out] - y_pre_co.loc[hold_out].dot(model.w_))

        #The weights applied to the data
        y0_hat = y_post_co.dot(model.w_)

        #Appending the weights so we can reference them later
        block_weights.append(model.w_.round(3))

        return (y_post_tr - y0_hat) - bias_hat

    for block in blocks:
        debiased_effects.append(fold_effect(block))

    results = {
        'debiased_effects': pd.DataFrame(debiased_effects).T,
        'block_weights': block_weights
    }

    return results

In [ ]:
debiasingResults = debiased_sc_atts(y_pre_co,
                                    y_pre_tr_variable,
                                    y_post_co,
                                    y_post_tr_variable,
                                    K=6)
deb_atts = debiasingResults["debiased_effects"]
block_weights = debiasingResults["block_weights"]

In [ ]:
#Plots
plt.plot(deb_atts.mean(axis=1).index, deb_atts.mean(axis=1).values, label="Debiased (Cross Validated) Effect")
plt.plot(y_post_tr_variable.index, y_post_tr_variable-y0_hat_cov, color='black', linestyle='dashed', alpha=0.5, label="Original Effect")

#Indicative Lines
plt.axvline(datetime.datetime(2024,4,10), color = 'black', linewidth=0.5)
plt.axhline(0, color = 'black', linewidth=0.5)

plt.legend()
plt.tight_layout()
plt.title("Treatment effect: Cross validation versus overfitting")
plt.xlim([datetime.datetime(2024,4,10), datetime.datetime(2024,5,21)])
plt.ylim([-0.1,0.1])
plt.tick_params(axis='x', labelrotation=45)

# plt.show()

## Inference

In [ ]:
atts_k = deb_atts.mean(axis=0).values
att = np.mean(atts_k)

# print(f"Treatment effect of the treated using K={len(atts_k)}:\n")
# print("atts_k:", atts_k)
# print("ATT:", att)

K = len(atts_k)
T0 = len(y_pre_co)
T1 = len(y_post_co)
block_size = min(np.floor(T0/K), T1)
se_hat=np.sqrt(1+((K*block_size)/T1))*np.std(atts_k, ddof=1)/np.sqrt(K)
print("SE:", se_hat)

from scipy.stats import t
alpha = 0.1
# [att - t.ppf(1-alpha/2, K-1)*se_hat, att + t.ppf(1-alpha/2, K-1)*se_hat]

In [ ]:
#Plots
plt.plot(deb_atts.mean(axis=1).index, deb_atts.mean(axis=1).values, label="Debiased (Cross Validated) Effect")
# plt.plot(y_post_tr_variable.index, y_post_tr_variable-y0_hat_cov, color='black', linestyle='dashed', alpha=0.5, label="Original Effect")

#Indicative Lines
plt.axvline(datetime.datetime(2024,4,10), color = 'black', linewidth=0.5)
plt.axhline(0, color = 'black', linewidth=0.5)
plt.axhline(att, color = 'tab:orange',linestyle='dashed', linewidth=1, label="Average Estimated Effect")

plt.legend()
plt.tight_layout()
plt.title("Treatment effect")
plt.xlim([datetime.datetime(2024,4,10), datetime.datetime(2024,5,21)])
plt.ylim([-0.1,0.1])
plt.axhspan(att - t.ppf(1-alpha/2, K-1)*se_hat, att + t.ppf(1-alpha/2, K-1)*se_hat, alpha=0.05, color='red')
plt.tick_params(axis='x', labelrotation=45)
plt.savefig("Effect_withInference.svg")
# plt.show()

In [ ]:
#Creating a dataframe for the weights for each holdout set
weightsFrame = pd.DataFrame(columns=y_pre_co.columns)
weightsFrame.index.names = ["holdOutSet"]
for i in range(len(block_weights)):
  weightsFrame.loc[i] = (block_weights[i])
# display(weightsFrame)


# Get unique column names and holdOutSet values
columns = weightsFrame.columns
holdout_sets = weightsFrame.index.unique()

# Set up the plot
fig, ax = plt.subplots(figsize=(10, 5))

# Set width of bars and positions of the bars on the x-axis
bar_width = 0.1
r = np.arange(len(columns))

# Create a color map for holdOutSet
color_map = plt.colormaps.get_cmap('Pastel1')
colors = [color_map(i/len(holdout_sets)) for i in range(len(holdout_sets))]

# Plot bars for each holdOutSet
for i, holdout in enumerate(holdout_sets):
    ax.bar(r + i * bar_width, weightsFrame.loc[holdout], width=bar_width,
           label=holdout, color=colors[i])

ax.set_ylabel('Weight')
ax.set_title('Weights of the donor pool units for each cross validation set')
ax.set_xticks(r + bar_width * (len(holdout_sets) - 1) / 2)
ax.set_xticklabels(columns, rotation=360-35, ha='left')
# ax.legend(title='Holdout Set')
plt.tight_layout()

plt.savefig("debiasedDist.svg")
# plt.show()

# SankeyDiagram of the Dataflow

In [ ]:
# sankeyData

In [ ]:
# # Define the nodes
# node_labels = ["Complete Set", "Filter on Date", "Filter on Device_type", "Filter by manual exclusion", "Removed", "Used Data"]
# node_colors = ["#1f77b4", "#ff7f0e", "#ff7f0e", "#ff7f0e", "#d62728", "#2ca02c"]

# # Define the links (source, target, value)
# links = [
#     #DateFilter
#     ("Complete Set", "Filter on Date", sankeyData['complete']),
#     ("Filter on Date", "Removed", sankeyData['complete']-sankeyData['datefilter']),
#     #DeviceType Filter
#     ("Filter on Date", "Filter on Device_type", sankeyData['datefilter']),
#     ("Filter on Device_type", "Removed", sankeyData['datefilter']-sankeyData['deviceTypeFilter']),
#     #Exclusion
#     ("Filter on Device_type", "Filter by manual exclusion", sankeyData['deviceTypeFilter']),
#     ("Filter by manual exclusion", "Removed", sankeyData['deviceTypeFilter']-sankeyData['manualExclusion']),
#     #Finally Used
#     ("Filter by manual exclusion", "Used Data", sankeyData['manualExclusion'])
# ]

# # Prepare data for Plotly
# source_indices = [node_labels.index(link[0]) for link in links]
# target_indices = [node_labels.index(link[1]) for link in links]
# values = [link[2] for link in links]

# # Create the Sankey diagram
# fig = go.Figure(data=[go.Sankey(
#     node = dict(
#       pad = 15,
#       thickness = 20,
#       line = dict(color = "black", width = 0.5),
#       label = node_labels,
#       color = node_colors
#     ),
#     link = dict(
#       arrowlen=0,
#       source = source_indices,
#       target = target_indices,
#       value = values
#   ))])

# # Update the layout
# fig.update_layout(title_text="Data flow", font_size=11)

# fig.update_layout(
#     # Adjust overall font size here
#     font=dict(size=14),
#     # Increase top margin to accommodate labels above nodes
#     margin=dict(t=50)
# )

# # Show the plot
# fig.show()

In [ ]:
#Altered Plotly plot to only report on percentages

# Define the nodes
node_labels = ["Complete Set", "Filter on Date", "Filter on Device_type", "Filter by manual exclusion", "Removed", "Used Data"]
node_colors = ["#1f77b4", "#ff7f0e", "#ff7f0e", "#ff7f0e", "#d62728", "#2ca02c"]

# Define the links (source, target, value)
links = [
    #DateFilter
    ("Complete Set", "Filter on Date", sankeyData['complete']),
    ("Filter on Date", "Removed", sankeyData['complete']-sankeyData['datefilter']),
    #DeviceType Filter
    ("Filter on Date", "Filter on Device_type", sankeyData['datefilter']),
    ("Filter on Device_type", "Removed", sankeyData['datefilter']-sankeyData['deviceTypeFilter']),
    #Exclusion
    ("Filter on Device_type", "Filter by manual exclusion", sankeyData['deviceTypeFilter']),
    ("Filter by manual exclusion", "Removed", sankeyData['deviceTypeFilter']-sankeyData['manualExclusion']),
    #Finally Used
    ("Filter by manual exclusion", "Used Data", sankeyData['manualExclusion'])
]

# Prepare data for Plotly
source_indices = [node_labels.index(link[0]) for link in links]
target_indices = [node_labels.index(link[1]) for link in links]
values = [link[2] for link in links]

# Calculate percentages of the complete set
percentages = [value / sankeyData['complete'] * 100 for value in values]

# Calculate percentages of the previous step
previous_step_percentages = []
for i, (source, target, value) in enumerate(links):
    if i == 0:  # First link is always 100% of the previous step
        previous_step_percentages.append(100)
    else:
        previous_value = next(link[2] for link in links if link[1] == source)
        previous_step_percentages.append(value / previous_value * 100)

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = node_labels,
      color = node_colors,
      hovertemplate='%{label}<extra></extra>'
    ),
    link = dict(
      arrowlen=0,
      source = source_indices,
      target = target_indices,
      value = values,
      hovertemplate='%{source.label} → %{target.label}<br>' +
                    'Percentage of complete set: %{customdata[0]:.2f}%<br>' +
                    'Percentage of previous step: %{customdata[1]:.2f}%<extra></extra>',
      customdata=list(zip(percentages, previous_step_percentages))
  ))])

# Update the layout
fig.update_layout(title_text="Data flow", font_size=11)
fig.update_layout(
    # Adjust overall font size here
    font=dict(size=14),
    # Increase top margin to accommodate labels above nodes
    margin=dict(t=50)
)

# Show the plot
fig.show()

# References
> Facure, M. (2023). _Causal inference in Python: Applying Causal Inference in the Tech Industry_ (First Edition). O’Reilly.

> Abadie, A. (2021). Using Synthetic Controls: Feasibility, Data Requirements, and Methodological Aspects. _Journal of Economic Literature_, _59_(2), 391–425.

>


